In [1]:
import cv2
import numpy as np
import dlib
import pandas as pd
import os

In [2]:
detector = dlib.get_frontal_face_detector() #fACE dETECTIOR
predictor = dlib.shape_predictor("data/shape_predictor_68_face_landmarks.dat") # predict 해주는애

In [3]:
def get_rect(shape):
    rw = 0
    rh = 0
    rx = 65535
    ry = 65535
    for (x,y) in shape:
        rw = max(rw,x)
        rh = max(rh,y)
        rx = min(rx,x)
        ry = min(ry,y)
    return (rx,ry,rw-rx,rh-ry)

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [ ]:
inpDir = 'data/dataset/dataset'
ratio = 1.0/2.0
linpDir = os.listdir(inpDir)
pi = 0
for personStr in linpDir:
    if personStr == ".DS_Store" :
        continue
    pi += 1
    print("person: %s [%d/%d]" % (personStr,pi,len(linpDir) - 1))
    personFolder = '%s/%s/words' % (inpDir,personStr)
    lpersonFolder = os.listdir(personFolder)
    
    for sentenceId in lpersonFolder:
        sentenceFolder = '%s/%s' % (personFolder,sentenceId)
        
        for sentenceId2 in os.listdir(sentenceFolder):
            sentenceFolder2 = '%s/%s/%s' % (personFolder,sentenceId,sentenceId2)
            
            for frame in os.listdir(sentenceFolder2):
                file = "%s/%s" % (sentenceFolder2,frame)
                if(not os.path.isfile(file)):
                    print("%s does not exist" % file)
                    sys.exit(1)
                if(frame[0:5] != "color"):
                    continue
                
                frame = frame[6:-4]
                image = cv2.imread(file)
                img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                #speed up detector by resizing first
                img2 = cv2.resize(img, dsize=(2560, 1920), interpolation=cv2.INTER_AREA)

                #detects whole face
                rects = detector(img2, 1)
                if len(rects) == 0:
                    print("error finding head at file: %s" % file)
                    continue
                    
                for face in rects:
                    crop_face = img2.copy()
                    crop_face = crop_face[face.top(): face.bottom(), face.left(): face.right()]
                    #rects[0] = dlib.scale_rect(rects[0],1/ratio)
                    shape = predictor(img2,rects[0])
                    shape = shape_to_np(shape)
                    #indices from 48 to 67 are for mouth
                    shape = [shape[x] for x in range(48,68)]
                    crop_lip = img2.copy()
                    (x, y, w, h) = get_rect(shape)
                    crop_lip = crop_lip[y: y+h, x:x+w]
                    
                    for (i, (x_i, y_i)) in enumerate(shape):
                        cv2.circle(image, (x_i, y_i), 1, (255, 0, 0), -1)
                    
                    (x, y, w, h) = get_rect(shape)
                
                    for s in shape:
                        s[0] -= x
                        s[1] -= y
                
                cropl_path = 'data/cropped/%s/%s/%s/crop_lip'%(personStr, sentenceId, sentenceId2)
                cropf_path = 'data/cropped/%s/%s/%s/crop_face'%(personStr, sentenceId, sentenceId2)
                
                try:
                    os.makedirs(cropl_path)
                    os.makedirs(cropf_path)
                except FileExistsError as e:
                    pass
            
                mouth_name = f'mouth_{frame}.jpg'
                face_name = f'face_{frame}.jpg'
                mouth_path = cropl_path + '/' + mouth_name
                face_path = cropf_path + '/' + face_name
        
                # save images by frame
                cv2.imwrite(face_path, crop_face)
                cv2.imwrite(mouth_path, crop_lip)
                #cv2.imwrite(face_path, crop_face)
            
        #for now just one sentence
        #if SHORT_TEST:
        #    break
    #for now just one person
    #if SHORT_TEST:
    #    break

person: F01 [1/14]
